<a href="https://colab.research.google.com/github/gmxavier/FAT0413150/blob/main/projects/team_work_2020_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TEAM WORK 2020_2

## Instructions

- Your team must completed this notebook until the informed due date.

## Identification

### TEAM X 
<!---
Change X for the letter of your team.
-->
### MEMBERS
<!---
List the team members and their e-mails
-->
Name 1 - E-mail 1    
Name 2 - E-mail 2    
Name 3 - E-mail 3    


## Question 1

Based on the model assigned to your team, insert text cells (including text, pictures, tables, equations, ...) to describe it. Please take care with this description and make it as complete as you can.   

NOTE: The text cells must be inserted below this one.

## Question 2

Linearize the model equations described at Question 1 (by hand) around an operation point. Insert text cells to describe the linearization process step by step.

NOTE: The text cells must be inserted below this one.

## Question 3

Apply the Laplace transform on the linearized model equations obtained at Question 2. Develop the model's block diagram, identify all the inputs and outputs and obtain all the system transfer functions. Insert text cells to describe the development process step by step.

NOTE: The text cells must be inserted below this one.

## Question 4

Select just one control variable, its manipulated variable and list the disturbances variables. Based on this selection and a sound engineering analysis, define the form of the transfer functions of the actuator $G_{a}(s)$ and the sensor $G_{m}(s)$, including numeric values of their parameters. Develop the closed-loop block diagram for this control loop assuming a PID controller and its respective transfer function. Obtain the closed-loop transfer functions for this control application. Insert text cells to describe the development process step by step.

NOTE: The text cells must be inserted below this one.

## Question 5

Based on the code templates given, adapt it to your model and insert the adapted code cells. Please take care with the code description (e.g. help, comments, measurement units). The code cells must include the setup, the model's update function, the code needed to obtain the transfer functions as well the code to obtain the process reaction curve.

NOTE: The code cells must be inserted below this one.

In [ ]:
# uncomment the 2 lines below if you had error
#!pip install control
#!pip install slycot
import numpy as np
import matplotlib.pyplot as plt
import control as ct
import slycot # to MIMO systems

In [ ]:
def berg_update(t, x, u, params={}):
  """Insulin dynamics based on Bergman minimal model.

    Parameters
    ----------
    x : array
         System state: G, the deviation of the blood glucose concentration from
                       basal levels measured in mg dL−1
                       I, the deviation of the blood insulin concentration from 
                       basal levels measured in mU L−1
                       X, a proportionality variable that describes the insulin 
                       concentration in a remote compartment measured in mU L−1
    u : array
         System input: U, the insulin input from an external source and into the
                       bloodstream, measured in mU min−1
                       D, a disturbance variable representing the intake of 
                       glucose from external sources, typically food, measured 
                       in mg dL−1min−1

    Returns
    -------
    y : array
        The time derivatives of G, I and X

    """

  # Set up the system parameters
  Gb = params.get('Gb', 81.)      # mg/dL
  Ib = params.get('Ib', 15.)      # mU/L
  P1 = params.get('P1', 0.0287)   # 1/min
  P1_s = params.get('P1_s', 0.02) # 1/min
  P2 = params.get('P2', 0.0287)   # 1/min 
  P3 = params.get('P3', 5.035e-5) # L/mU/min^2  
  n = params.get('n', 5/54)       # 1/min  
  V1 = params.get('V1', 12)       # L  

  # Define variables for patient state and inputs
  G = x[0]                           # blood glucose concentration
  I = x[1]                           # blood insulin concentration
  X = x[2]                           # insulin concentration in a remote 
                                     # compartment
  U = u[0]                           # insulin input
  D = u[1]                           # intake of glucose

  # Define the ODEs
  dGdt = -P1*G - X*(G+Gb) + D
  dIdt = -n*(I+Ib) + U/V1
  dXdt = -P2*X + P3*I

  return [dGdt, dIdt, dXdt]

In [ ]:
# Obtain the transfer functions of the input/output system for the bergman minimal model
bergman = ct.NonlinearIOSystem(
    berg_update, None, name='bergman',
    inputs = ('U', 'D'), outputs = ('G', 'I', 'X'), states=('G', 'I', 'X'))

bergman_linearized = bergman.linearize(x0=[0, 0, 0], u0=[0, 0])

bergman_tf = ct.ss2tf(bergman_linearized)

bergman_tf

In [ ]:
# Obtain the process reaction curve response
# Define the simulation time
T0 = 0
T1 = 400
dt = 0.01
nt = int (T1/dt) + 1 # Number of points of sim time
T = np.linspace(T0, T1, nt)

# Perform the simulation
Ga = 1
Gm = 1
T, yout = ct.step_response(Ga*Gp*Gm, T)

# Plot the process reaction curve response
plt.close ('all')
fig_width_cm = 24
fig_height_cm = 18
plt.figure(1, figsize=(fig_width_cm /2.54 , fig_height_cm /2.54))
plt.subplot(2, 1, 1)
plt.plot(T, yout, 'blue')
plt.ylabel('G [mg/dL]')
plt.grid()
plt.legend(labels =('y',))
plt.subplot(2, 1, 2)
plt.plot(T, [1]*len(T), 'green')
plt.ylabel('U [mU min−1]')
plt.xlabel('t [s]')
plt.grid()
plt.legend(labels =('u', ))

## Question 6

Use the process reaction curve obtained at Question 5 to develop a FOLPD (or a IPD) approximation of the process reaction curve transfer function $G_{PRC}(s)$. Insert text cells to describe the development process step by step.

NOTE: FOLPD means first-order lag plus time delay and IPD means integral plus time delay.

## Question 7

Based on the template given, code the tuning rules assigned to your team. Insert one code cell for each tuning rule. Please take care with the code description (e.g. help, comments, measurement units) and adapt all code to the respective tuning rule.

NOTE: The code cells must be inserted below this one.

In [3]:
def callender356_folpd_pi(Km, taum, Tm, rule):
  r'''Calculates the PI controller parameters for a FOLPD process reaction 
  curve using the rule of Callender et al. (1935/6).

  Parameters
  ----------
  Km : float
       Static gain of the process reaction curve, [-] 
  taum : float
       Dead time of the process reaction curve, [min]
  Tm : float
       Time constant (lag) of the process reaction curve, [min]
  rule : integer
       If rule = 1, the rule gives decay ratio = 0.015 and period of decaying 
       oscillation = 5.10*taum.
       If rule = 2, the rule gives decay ratio = 0.043 and period of decaying 
       oscillation = 6.28*taum.

  Returns
  -------
  Kc : float
       Proportional gain, [-]

  Ti : float
       Integral time, [min]

  Notes
  -----
  Applicable to taum/Tm = 0.3.

  Example
  --------

  >>> callender356_folpd_pi(Km=1, taum=3, Tm=10, rule=1)
  [0.18933333333333333, 10.92]  

  Reference
  ----------
  .. [1] O’Dwyer, A. Handbook of PI and PID Controller Tuning Rules. London:
     Imperial College Press, 2009.
  '''
  if rule not in [1,2]:
    return []
  # Rule 1 (Table 9, p. 30)
  if rule == 1:
    Kc = 0.568/(Km*taum)
    Ti = 3.64*taum
    return [Kc, Ti]
  # Rule 2 (Table 9, p. 30)
  if rule == 2:
    Kc = 0.690/(Km*taum)
    Ti = 2.45*taum
    return [Kc, Ti]

## Question 8

Based on the process reaction curve transfer function $G_{PRC}(s)$ obtained at Question 6 and the functions coded at Question 7, choose one tuning rule function coded by each team member and obtain the respective values of the controller parameters. Provide a summary table listing the tuning rules and the controlle parameters. 

## Question 9

Choose two sets of parameter values from the ones obtained at Question 8, one for PI and other for PID controller. Obtain the closed-loop transfer functions for servo control using the process reaction curve transfer function obtained at Question 6. Check the stability of the closed-loop by calculating their poles.

NOTE: The code cells must be inserted below this one.

## Question 10

For each controller parameters sets obtained at Question 8 and the template code given, simulate two closed-loop responses, one for a step in the setpoint and the other for a step in one of the disturbances. Comment the obtained responses regarding the performance of the tuning rules and indicate the best tuning rule for the servo control and for the regulator control.

NOTE: Define the step amplitude values based on a sound engineering analysis. The code cells must be inserted below this one.